In [1]:
from packages.transcripts import transcribe_with_timestamps
from packages.PII_detection import detect_pii_entities
from packages.mapTimeStamps import map_pii_to_timestamps
from packages.saveRedactedAudo import redact_audio

In [2]:
input_audio_path = "inputs\in-9872019796-1001-20250530-154113-1748599873.42156.wav"

<>:1: SyntaxWarning: invalid escape sequence '\i'
<>:1: SyntaxWarning: invalid escape sequence '\i'
C:\Users\anipr\AppData\Local\Temp\ipykernel_13104\3500974511.py:1: SyntaxWarning: invalid escape sequence '\i'
  input_audio_path = "inputs\in-9872019796-1001-20250530-154113-1748599873.42156.wav"


### 🎙️ Step 1: Transcribe Hindi Audio with Word-Level Timestamps

This function uses Azure's Speech SDK to transcribe a WAV audio file in Hindi (`hi-IN`) and extract:

- Recognized speech as text.
- Word-level timestamps including start and end time (in seconds).

The results are collected and returned in a structured dictionary with `text` and `chunks`.


In [3]:
transcript = transcribe_with_timestamps(input_audio_path)
print(transcript)

{'text': 'ਹੈਲੋ। ਹੈਲੋ। ਹੈਲੋ ਹਾਂ ਜੀ ਸਤਿ ਸ੍ਰੀ ਅਕਾਲ ਜੀ ਕੁਲਦੀਪ ਸਿੰਘ ਜੀ ਗੱਲ ਕਰ ਰਹੇ ਹੋ। ਕੁਲਬੀਰ ਜੀ ਮੈਂ ਕਲਤਿਬਿਟ ਕੰਪਨੀ ਪਟਿਆਲੇ ਤੋਂ ਗੱਲ ਕਰਦੀ ਆਂ। ਹੁਣੇ ਉਹਨਾਂ ਨੇ ਕਾਰਬਨ ਕ੍ਰੇਡਿਟ ਪ੍ਰੋਗ੍ਰਾਮ ਬਾਰੇ ਜਾਣਕਾਰੀ ਦਿੱਤੀ ਹੋਣੀ ਜਿਹੜੇ ਖੇਤਾਂ ਦੇ ਵਿੱਚ ਪਾਈਪ ਲੱਗਦੇ ਨੇ ਸਿਰ ਪਾਣੀ ਦਾ ਸੁਨੇਹਾ ਲਾਉਂਦੇ ਨੇ ਆਪਾਂ ਨੂੰ। ਠੀਕ ਐ ਜੀ ਤੇ ਮੈਂ ਤੁਹਾਨੂੰ ਇੱਕ ਤਰ੍ਹਾਂ ਦੀ ਕੰਪਨੀ ਵੱਲੋਂ ਤੁਹਾਨੂੰ ਵੈਲਕਮ ਕਾਲ ਹੈ ਤੇ ਦੂਜਾ ਜੇ ਤੁਹਾਨੂੰ ਇੱਕ ਪ੍ਰੋਜੈਕਟ ਬਾਰੇ ਕੁਝ ਵੀ ਸਮਝ ਨਹੀਂ ਆਇਆ ਜਾਣਨਾ ਚਾਹੁੰਦੇ ਹੋ ਤਾਂ ਤੁਸੀਂ ਮੈਨੂੰ ਪੁੱਛ ਸਕਦੇ ਹੋ। ਤੁਹਾਨੂੰ ਸਮਝ ਆਇਆ ਕਿ ਆਪਾਂ ਜੀ ਸੈਂਸਰ ਯਾ ਪਾਈਪ ਤੁਹਾਡੇ ਖੇਤਾਂ ਚ ਕਿਉਂ ਲਾਉਣੀ? ਨਹੀਂ ਜੀ ਓਕੇ ਕੁਲਦੀਪ ਸਿੰਘ ਜੀ ਇਹ ਹੈ ਕਿ ਜਿਵੇਂ ਆਪਾਂ ਕਈ ਵਾਰੀ ਝੋਨੇ ਦੀ ਫਸਲ ਜਦੋਂ ਲਾ ਦਿੰਦੇ ਐ ਨਾ ਉਹ ਤੋਂ ਬਾਅਦ ਸਤ੍ਹਾ ਨੂੰ ਸੁੱਕਾ ਦੇਖ ਕੇ ਪਤਾ ਨਹੀਂ ਲਾ ਦਿੰਦੇ ਆ ਠੀਕ ਐ ਪਹਿਲਾਂ ਮੈਨੂੰ ਹੀ ਦੱਸੋਂਗੇ ਕਿ ਸੈਂਸਰ ਕੰਮ ਕੀ ਕਰਦੇ ਨੇ ਇਹ ਸੈਂਸਰ ਨਾ ਸਿਰ ਪੰਜ ਇੰਚ ਦੇ ਡਾਇਮੀਟਰ ਦੇ ਹੁੰਦੇ ਨੇ ਖੇਤ ਦੀ ਵੱਟ ਤੇ ਲੱਗ ਜਾਂਦੇ ਨੇ ਥੋੜੀ ਫ਼ਸਲ ਨੂੰ ਕੋਈ ਨੁਕਸਾਨ ਨਹੀਂ ਕੀਤਾ ਗਿਆ। ਪਾਣੀ ਦੀ ਮਾਤਰਾ ਦੀ ਲੋੜ ਹੋਊਗੀ ਪਾਣੀ ਦੀ ਲੋੜ ਹੋਊਗੀ ਮਸਜਿਦਏਗਾ ਵੀ ਕੁਲਦੀਪ ਸਿੰਘ ਜੀ ਥੋਡੇ ਖੇਤ ਅੰਦਰ ਨਮੀ ਦੀ ਮਾਤਰਾ ਘਟ ਗਈ ਹੈ। ਕਿਰਪਾ ਕਰਕੇ ਮੋਟਰ ਚਲਾ ਦੋ। ਠੀਕ ਹੈ ਜਦੋਂ ਪਾਣੀ ਦੀ ਮਾਤਰਾ ਪੂਰੀ ਹੋ ਚੁੱਕੀ ਹੈ। ਤੁਹ

### 🕵️ Step 2: Detect PII Entities in Hindi Text

This function uses the Azure client to recognize PII entities in a Hindi document. It:

- Calls Azure's `recognize_pii_entities()` method with language set to `"hi"` (Hindi).
- Prints redacted text and detailed info for each detected entity.
- Returns a list of all PII words detected in the input.


In [4]:
asr_text = [transcript['text']]
pii_result = detect_pii_entities(asr_text)
pii_words = []
for doc in pii_result:
    print("Redacted Text: {}".format(doc.redacted_text))
    for entity in doc.entities:
        if entity.confidence_score > 0.6 and entity.category in ['Person', 'Organization', 'PhoneNumber', 'Address', 'Location']:
            print("Entity: {}".format(entity.text))
            pii_words.append(entity.text)
            print("\tCategory: {}".format(entity.category))
            print("\tConfidence Score: {}".format(entity.confidence_score))
            print("\tOffset: {}".format(entity.offset))
            print("\tLength: {}".format(entity.length))

Redacted Text: ਹੈਲੋ। ਹੈਲੋ। ਹੈਲੋ ਹਾਂ ਜੀ ਸਤਿ ਸ੍ਰੀ ਅਕਾਲ ਜੀ *********** ਜੀ ਗੱਲ ਕਰ ਰਹੇ ਹੋ। ****** ਜੀ ਮੈਂ ************* ਪਟਿਆਲੇ ਤੋਂ ਗੱਲ ਕਰਦੀ ਆਂ। **** ਉਹਨਾਂ ਨੇ ਕਾਰਬਨ ਕ੍ਰੇਡਿਟ ਪ੍ਰੋਗ੍ਰਾਮ ਬਾਰੇ ਜਾਣਕਾਰੀ ਦਿੱਤੀ ਹੋਣੀ ਜਿਹੜੇ ਖੇਤਾਂ ਦੇ ਵਿੱਚ ਪਾਈਪ ਲੱਗਦੇ ਨੇ ਸਿਰ ਪਾਣੀ ਦਾ ਸੁਨੇਹਾ ਲਾਉਂਦੇ ਨੇ ਆਪਾਂ ਨੂੰ। ਠੀਕ ਐ ਜੀ ਤੇ ਮੈਂ ਤੁਹਾਨੂੰ ਇੱਕ ਤਰ੍ਹਾਂ ਦੀ ਕੰਪਨੀ ਵੱਲੋਂ ਤੁਹਾਨੂੰ ਵੈਲਕਮ ਕਾਲ ਹੈ ਤੇ ਦੂਜਾ ਜੇ ਤੁਹਾਨੂੰ ਇੱਕ ਪ੍ਰੋਜੈਕਟ ਬਾਰੇ ਕੁਝ ਵੀ ਸਮਝ ਨਹੀਂ ਆਇਆ ਜਾਣਨਾ ਚਾਹੁੰਦੇ ਹੋ ਤਾਂ ਤੁਸੀਂ ਮੈਨੂੰ ਪੁੱਛ ਸਕਦੇ ਹੋ। ਤੁਹਾਨੂੰ ਸਮਝ ਆਇਆ ਕਿ ਆਪਾਂ ਜੀ ਸੈਂਸਰ ਯਾ ਪਾਈਪ ਤੁਹਾਡੇ ਖੇਤਾਂ ਚ ਕਿਉਂ ਲਾਉਣੀ? ਨਹੀਂ ਜੀ ਓਕੇ *********** ਜੀ ਇਹ ਹੈ ਕਿ ਜਿਵੇਂ ਆਪਾਂ ਕਈ ਵਾਰੀ ਝੋਨੇ ਦੀ ਫਸਲ ਜਦੋਂ ਲਾ ਦਿੰਦੇ ਐ ਨਾ ਉਹ ਤੋਂ ਬਾਅਦ ਸਤ੍ਹਾ ਨੂੰ ਸੁੱਕਾ ਦੇਖ ਕੇ ਪਤਾ ਨਹੀਂ ਲਾ ਦਿੰਦੇ ਆ ਠੀਕ ਐ ਪਹਿਲਾਂ ਮੈਨੂੰ ਹੀ ਦੱਸੋਂਗੇ ਕਿ ਸੈਂਸਰ ਕੰਮ ਕੀ ਕਰਦੇ ਨੇ ਇਹ ਸੈਂਸਰ ਨਾ ਸਿਰ ਪੰਜ ਇੰਚ ਦੇ ਡਾਇਮੀਟਰ ਦੇ ਹੁੰਦੇ ਨੇ ਖੇਤ ਦੀ ਵੱਟ ਤੇ ਲੱਗ ਜਾਂਦੇ ਨੇ ਥੋੜੀ ਫ਼ਸਲ ਨੂੰ ਕੋਈ ਨੁਕਸਾਨ ਨਹੀਂ ਕੀਤਾ ਗਿਆ। ਪਾਣੀ ਦੀ ਮਾਤਰਾ ਦੀ ਲੋੜ ਹੋਊਗੀ ਪਾਣੀ ਦੀ ਲੋੜ ਹੋਊਗੀ ਮਸਜਿਦਏਗਾ ਵੀ *********** ਜੀ ਥੋਡੇ ਖੇਤ ਅੰਦਰ ਨਮੀ ਦੀ ਮਾਤਰਾ ਘਟ ਗਈ ਹੈ। ਕਿਰਪਾ ਕਰਕੇ ਮੋਟਰ ਚਲਾ ਦੋ। ਠੀਕ ਹੈ ਜਦੋਂ ਪਾਣੀ ਦੀ ਮਾਤਰਾ ਪੂਰੀ ਹੋ ਚੁੱਕੀ ਹੈ

### 🧠 Step 3: Map PII Words to Audio Word Timestamps

This function attempts to align each detected PII word to a timestamped word chunk in the ASR output using:

- Fuzzy string matching (`SequenceMatcher`) to allow for slight mismatches.
- Returns a list of timestamps corresponding to each PII word.

This is crucial for knowing **where** in the audio to apply redaction.


In [5]:
pii_segments = map_pii_to_timestamps(pii_words, transcript['chunks'], threshold=0.6)
print(pii_segments)

[(10.92, 11.16), (13.03, 13.55), (13.75, 14.15), (45.87, 46.83), (74.63, 75.03), (83.99, 84.43)]


In [6]:
len(pii_segments)

6

### 🔇 Step 4: Mute Audio Segments Containing PII

This function takes the original audio file and the timestamps of detected PII, and:

- Loads the audio using `torchaudio`.
- Silences the waveform between each PII segment (sets amplitude to zero).
- Saves the redacted audio file to `output_audio/` directory, replacing `"in-"` with `"out-"` in the filename.


In [7]:
redact_audio(input_audio_path,pii_segments)

✅ Redacted audio saved at: output_audio\out-9872019796-1001-20250530-154113-1748599873.42156.wav
